In [2]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
import os
from splinter import Browser
import pandas as pd
import time
from webdriver_manager.chrome import ChromeDriverManager
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\BZ Northwestern BC\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database and collection  from Eli's Stu_Reddit_Scraper for 12.2.4 that he slacked to us at 7:51 on 3/16
db = client.mars_news_db
collection = db.articles

print(collection)
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mars_news_db'), 'articles')


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'mars_news_db'), 'articles')

Step 1 - Scraping:  NASA Mars News

In [4]:
# URL of page to be scraped
url = 'https://redplanetscience.com/'

# Retrieve page with the requests module
# response = requests.get(url)
browser.visit(url)
html = browser.html

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

#print(soup.prettify())
#print(soup)

news_title =soup.find("div", "content_title").get_text()
news_body = soup.find("div", "article_teaser_body").get_text()
news_title = news_title.replace(news_body, '')

print(news_title)
print()
print(news_body)


NASA Wins 4 Webbys, 4 People's Voice Awards

Winners include the JPL-managed "Send Your Name to Mars" campaign, NASA's Global Climate Change website and Solar System Interactive.


In [5]:
# Retrieve the parent divs for all articles
# results = soup.find_all('div', class_='content_title')
# print(results)

results = soup.find('div', class_='content_title').get_text()
print(results)


results2 = soup.find('div', class_='article_teaser_body').get_text()
print(results2)

# loop over results to get article data
for result in results:
    # scrape the article header 
    header = result.find('h1', class_='article-item__headline').text
    
    # scrape the article subheader
    subheader = result.find('h2', class_='article-item__subheader').text
    
    # scrape the datetime
    datetime = result.find('span', class_='article-item__date')['data-date'] 
    
    # get only the date from the datetime
    date = datetime.split('T')[0]
    
    # print article data
    print('-----------------')
    print(header)
    print(subheader)
    print(date)

    # Dictionary to be inserted into MongoDB
    post = {
        'header': header,
        'subheader': subheader,
        'date': date,
    }

    # Insert dictionary into MongoDB as a document
    collection.insert_one(post)

print()
print("Need to work on this later")

NASA Wins 4 Webbys, 4 People's Voice Awards
Winners include the JPL-managed "Send Your Name to Mars" campaign, NASA's Global Climate Change website and Solar System Interactive.


TypeError: find() takes no keyword arguments

In [6]:
# Display the MongoDB records created above
articles = db.articles.find()
print(articles)

for article in articles:
    print(f'article is {article}')

print()
print("Need to work on this later")


Need to work on this later


In [7]:
browser.quit()

Step 1 - Scraping:  JPL Mars Space Images - Featured Image

In [10]:
browser = Browser("chrome", **executable_path, headless=False)
url = 'https://spaceimages-mars.com/'

# Retrieve page with the visit module
browser.visit(url)
html = browser.html

# Click on "FULL IMAGE" button
browser.links.find_by_partial_text('FULL IMAGE').click()

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

print(soup.prettify())
print(soup)

fade_in_image = soup.find_all("img", "headerimage fade-in")
###news_body = soup.find("div", "article_teaser_body").get_text()
###news_title = news_title.replace(news_body, '')
print()
print("fade_in_image")
print(fade_in_image)
print()


# # Iterate through each image 
# for f_i_image in fade_in_image:
#     # Use Beautiful Soup's find() method to navigate and retrieve attributes
#     h3 = f_i_image.find('h3')
#     link = h3.find('a')
#     src_link = link['src']
#     ###title = link['title']
#     print('-----------')
#     print(src_link)
#     print('http://books.toscrape.com/' + src_link)

# link = fade_in_image.a['src']
# print(link)

# print()
# print("src")
# print(src)
# print()



featured_image_url = news_title

print()
print("featured_image_url")
print(featured_image_url)
print()

<html class="">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
  <!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
  <title>
   Space Image
  </title>
  <style type="text/css">
   .fancybox-margin{margin-right:17px;}
  </style>
 </head>
 <body>
  <div class="header">
   <nav class="navbar navbar-expand-lg">
    <a class="navbar-brand" href="#">
     <img id="logo" src="image/nasa.png"/>
     <span class="logo">
      Jet Propulsion Laboratory
     </span>
     <span class="logo1">
      California Institute of Technology
     </span>
    </a>
    <button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle

In [ ]:
browser.quit()

Step 1 - Scraping:  JPL Mars Space Images - Mars Facts

In [7]:
url = 'https://galaxyfacts-mars.com/'

tables = pd.read_html(url)
tables
# print(type(tables)) #BZ saw it returns list type


[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [13]:
# Get Dataframe of first table
df = tables[0]
print(df.head())

# Convert df to html
html_table = df.to_html()
html_table
print(html_table)


                         0                1                2
0  Mars - Earth Comparison             Mars            Earth
1                Diameter:         6,779 km        12,742 km
2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
3                   Moons:                2                1
4       Distance from Sun:   227,943,824 km   149,598,262 km
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
      <th>2</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Mars - Earth Comparison</td>
      <td>Mars</td>
      <td>Earth</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Diameter:</td>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <

Step 1 - Scraping:  Mars Hemispheres

In [15]:

browser = Browser("chrome", **executable_path, headless=False)
url = 'https://marshemispheres.com/'

# Retrieve page with the visit module
browser.visit(url)
html = browser.html
print(html)

# Example: (from HW assignment)
# hemisphere_image_urls = [
#     {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#     {"title": "Cerberus Hemisphere", "img_url": "..."},
#     {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#     {"title": "Syrtis Major Hemisphere", "img_url": "..."},
# ]


# # Click on "FULL IMAGE" button
# browser.links.find_by_partial_text('FULL IMAGE').click()

# # Create BeautifulSoup object; parse with 'html.parser'
# soup = BeautifulSoup(html, 'html.parser')

# print(soup.prettify())
# print(soup)

# fade_in_image = soup.find_all("img", "headerimage fade-in")
# ###news_body = soup.find("div", "article_teaser_body").get_text()
# ###news_title = news_title.replace(news_body, '')
# print()
# print("fade_in_image")
# print(fade_in_image)
# print()


# # # Iterate through each image 
# # for f_i_image in fade_in_image:
# #     # Use Beautiful Soup's find() method to navigate and retrieve attributes
# #     h3 = f_i_image.find('h3')
# #     link = h3.find('a')
# #     src_link = link['src']
# #     ###title = link['title']
# #     print('-----------')
# #     print(src_link)
# #     print('http://books.toscrape.com/' + src_link)

# # link = fade_in_image.a['src']
# # print(link)

# # print()
# # print("src")
# # print(src)
# # print()



# featured_image_url = news_title

# print()
# print("featured_image_url")
# print(featured_image_url)
# print()

<html lang="en"><head>
      <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
      <link rel="stylesheet" type="text/css" href="css/jquery-ui.css">
      <title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
      <meta name="description" content="GUSS Astrogeology Science Center Astropedia search results.">
      <meta name="keywords" content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping">
      <meta http-equiv="X-UA-Compatible" content="IE=edge">
      <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1">
      <link rel="stylesheet" media="screen" href="css/main.css">
      <link rel="stylesheet" media="print" href="css/print.css">

      <link rel="icon" type="image/x-ico" href="#">
   </head>
   <body id="results">
      <header>
         <a href="#" style="float:right;margin-top:10px;" target="_blank">
         <img class="logo" height="60" src="images/usgs_logo_main

Step 2 - MongoDB and Flask Application